In [1]:
import sys
sys.path.append("..")
from utils.download_dataset import download_dataset, extract_zip

In [2]:
import numpy as np
import PIL

from fastai.vision.all import *
from pathlib import Path
from tqdm.auto import tqdm
from shutil import copyfile, rmtree

In [3]:
DATA_PATH = Path.cwd()/'data'
if not DATA_PATH.exists():
    DATA_PATH.mkdir(exist_ok=True)

fpath = download_dataset(dataset_name='MAGNETIC_TILE_SURFACE_DEFECTS', dest_dir=DATA_PATH)
fpath

File exists: /home/ubuntu/github/cv_fastai/segmentation/data/MAGNETIC_TILE_SURFACE_DEFECTS.zip


Path('/home/ubuntu/github/cv_fastai/segmentation/data/MAGNETIC_TILE_SURFACE_DEFECTS.zip')

In [4]:
dir_path = extract_zip(fpath)
dir_path

Directory exists: /home/ubuntu/github/cv_fastai/segmentation/data/MAGNETIC_TILE_SURFACE_DEFECTS


Path('/home/ubuntu/github/cv_fastai/segmentation/data/MAGNETIC_TILE_SURFACE_DEFECTS')

In [5]:
dir_path.ls()

(#7) [Path('/home/ubuntu/github/cv_fastai/segmentation/data/MAGNETIC_TILE_SURFACE_DEFECTS/MT_Uneven'),Path('/home/ubuntu/github/cv_fastai/segmentation/data/MAGNETIC_TILE_SURFACE_DEFECTS/MT_Fray'),Path('/home/ubuntu/github/cv_fastai/segmentation/data/MAGNETIC_TILE_SURFACE_DEFECTS/MT_Blowhole'),Path('/home/ubuntu/github/cv_fastai/segmentation/data/MAGNETIC_TILE_SURFACE_DEFECTS/images'),Path('/home/ubuntu/github/cv_fastai/segmentation/data/MAGNETIC_TILE_SURFACE_DEFECTS/MT_Crack'),Path('/home/ubuntu/github/cv_fastai/segmentation/data/MAGNETIC_TILE_SURFACE_DEFECTS/labels'),Path('/home/ubuntu/github/cv_fastai/segmentation/data/MAGNETIC_TILE_SURFACE_DEFECTS/MT_Break')]

In [6]:
if (dir_path/'MT_Free').exists():
    rmtree(dir_path/'MT_Free')

In [7]:
classes = []
for ii in (dir_path).ls():
    if ii.is_dir() and ii.stem.startswith('MT_'):
        classes.append(ii.stem)
classes

['MT_Uneven', 'MT_Fray', 'MT_Blowhole', 'MT_Crack', 'MT_Break']

In [8]:
classes_dict = {c:i+1 for i, c in enumerate(classes)}
classes_dict

{'MT_Uneven': 1, 'MT_Fray': 2, 'MT_Blowhole': 3, 'MT_Crack': 4, 'MT_Break': 5}

In [9]:
img_fpaths = [get_files(dir_path/c, extensions='.jpg') for c in classes]
img_fpaths = [ii for sublist in img_fpaths for ii in sublist]
len(img_fpaths)

392

In [10]:
msk_fpaths = [get_files(dir_path/c, extensions='.png') for c in classes]
msk_fpaths = [ii for sublist in msk_fpaths for ii in sublist]
len(msk_fpaths)

392

In [11]:
assert len(img_fpaths) == len(msk_fpaths)

In [12]:
path_images = dir_path/'images'
if not path_images.exists():
    path_images.mkdir(exist_ok=True)
    
path_labels = dir_path/'labels'
if not path_labels.exists():
    path_labels.mkdir(exist_ok=True)

In [13]:
for msk_path in tqdm(msk_fpaths):
    c = msk_path.parent.parent.stem
    i = classes_dict[c]
    msk = np.array(Image.open(msk_path))
    msk[msk>0] = i
    Image.fromarray(msk).save(path_labels/msk_path.name)

  0%|          | 0/392 [00:00<?, ?it/s]

In [14]:
for img_fpath in tqdm(img_fpaths):
    copyfile(img_fpath, path_images/img_fpath.name)

  0%|          | 0/392 [00:00<?, ?it/s]

In [15]:
new_img_fpaths = get_files(path_images)
assert len(new_img_fpaths) == len(img_fpaths)